In [1]:
import h2o
import pandas as pd

In [2]:
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='30G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /home/tunguz/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpwz1e3mjo
  JVM stdout: /tmp/tmpwz1e3mjo/h2o_tunguz_started_from_python.out
  JVM stderr: /tmp/tmpwz1e3mjo/h2o_tunguz_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 16 days
H2O_cluster_name:,H2O_from_python_tunguz_s7jb5k
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,30 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
train = h2o.import_file("../input/train.csv")
test = h2o.import_file("../input/test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
train.head()

date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
2010-03-10 18:00:00,13.1,46,0.7578,1387.2,1087.8,1056,1742.8,1293.4,2.5,12,167.7
2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117,2.2,9.2,127.1
2010-03-10 21:00:00,11,62.4,0.7867,1321,929,1102.9,1536.5,1263.2,2.2,9.7,177.2
2010-03-10 22:00:00,11.9,59,0.7888,1272,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8
2010-03-10 23:00:00,11.2,56.8,0.7848,1220.9,697.5,1417.2,1462.6,949,1.2,4.4,88.1
2010-03-11 00:00:00,10.7,55.7,0.7603,1244.2,669.3,1491.2,1413,769.6,1.2,3.7,59.5
2010-03-11 01:00:00,10.3,57,0.7702,1181.4,631.7,1511.1,1359.7,715.4,1,3.4,63.9
2010-03-11 02:00:00,10.1,62.7,0.7648,1159.6,602.9,1610.6,1212.2,657.2,0.9,2.2,46.4
2010-03-11 03:00:00,10.5,59.6,0.7517,1030.2,521.7,1790.2,1148.6,491,0.6,1.6,43


In [6]:
test.head()

date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
2011-01-01 00:00:00,8,41.3,0.4375,1108.8,745.7,797.1,880,1273.1
2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714
2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878,693.7,941.9,1300.8
2011-01-01 03:00:00,5,52.3,0.4693,1139.7,916.2,725.6,1011,1283
2011-01-01 04:00:00,4.5,57.5,0.465,1022.4,838.5,871.5,967,1142.3
2011-01-01 05:00:00,4.5,53.7,0.4759,1004,745.5,914.2,989.1,973.8
2011-01-01 06:00:00,3.3,54.8,0.4636,940.9,738.2,816,896.8,1049.4
2011-01-01 07:00:00,3.2,60.7,0.4667,954.5,713.9,834.7,935.6,956.3
2011-01-01 08:00:00,2.5,65.7,0.4721,969.9,679.1,943.8,959.3,892
2011-01-01 09:00:00,3.9,57.8,0.4807,976.6,655.5,996,906,817.5


In [7]:
x = test.columns[1:]
y = 'target_benzene'

In [8]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=180, seed=47, max_runtime_secs=600)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [9]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210706_121036,1.328,1.15239,1.328,0.778898,0.0885665
StackedEnsemble_BestOfFamily_AutoML_20210706_121036,1.3617,1.16692,1.3617,0.787229,0.0891511
GBM_2_AutoML_20210706_121036,1.40113,1.18369,1.40113,0.79667,0.0907508
GBM_1_AutoML_20210706_121036,1.4109,1.18781,1.4109,0.802771,0.0910528
GBM_4_AutoML_20210706_121036,1.41644,1.19014,1.41644,0.801363,0.0901836
GBM_grid__1_AutoML_20210706_121036_model_9,1.42351,1.19311,1.42351,0.801139,0.0904276
GBM_grid__1_AutoML_20210706_121036_model_14,1.432,1.19666,1.432,0.797832,0.0900157
GBM_3_AutoML_20210706_121036,1.43479,1.19783,1.43479,0.804334,0.0902492
GBM_grid__1_AutoML_20210706_121036_model_8,1.43905,1.1996,1.43905,0.834375,0.0953742
GBM_grid__1_AutoML_20210706_121036_model_20,1.44322,1.20134,1.44322,0.807833,0.0906985


In [10]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20210706_121036

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.6663768024332314
RMSE: 0.8163190567622634
MAE: 0.5909431935709643
RMSLE: 0.0723768025476524
R^2: 0.9887428481573879
Mean Residual Deviance: 0.6663768024332314
Null degrees of freedom: 7110
Residual degrees of freedom: 7094
Null deviance: 420941.77180461504
Residual deviance: 4738.605442102708
AIC: 17329.788926279176

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 1.327998246019049
RMSE: 1.1523880622511884
MAE: 0.7788975275409336
RMSLE: 0.08856651447098952
R^2: 0.9775660289380245
Mean Residual Deviance: 1.327998246019049
Null degrees of freedom: 7110
Residual degrees of freedom: 7094
Null deviance: 421008.0039953752
Residual deviance: 9443.395527441457
AIC: 22233.34060429051


In [11]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [12]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission[y] = preds.as_data_frame().values.flatten()
sample_submission.to_csv(f'../submissions/h2o_submission_{y}_1.csv', index=False)